In [1]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper


In [6]:
class ResearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query to search the web or wikipedia for information")



class DdgResearchTool(BaseTool):
    name: str = "duckduckgo"
    description: str = "Search the web for information"
    args_schema: Type[BaseModel] = ResearchToolArgsSchema
    def _run(self, query: str) -> str:
        try:
            return DuckDuckGoSearchAPIWrapper().run(query)
        except Exception as e:
            return f"DDG Error: {e}"


class WikipediaResearchTool(BaseTool):
    name: str = "wikipedia"
    description: str = "Search the wikipedia for information"
    args_schema: Type[BaseModel] = ResearchToolArgsSchema
    def _run(self, query: str) -> str:
        try:
            return WikipediaAPIWrapper().run(query)
        except Exception as e:
            return f"Wikipedia Error: {e}"

llm = ChatOpenAI(temperature=0.1)

agent = initialize_agent(
    llm = llm,
    verbose = True,
    agent = AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors = True,
    tools = [DdgResearchTool(), WikipediaResearchTool()],
)

prompt = "Research about the XZ backdoor"

result = agent.invoke(prompt)

with open("research_result.txt", "w", encoding="utf-8") as f:
    f.write(result["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo` with `{'query': 'XZ backdoor'}`


DDG Error: 
Invoking: `wikipedia` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his f